In [1]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")

Using MPS


In [2]:
from transformers import BertForSequenceClassification, BertTokenizer, BertConfig

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
config = BertConfig.from_pretrained('bert-base-uncased')
config.num_labels = 6
config.problem_type = "multi_label_classification"
model = BertForSequenceClassification(config)
model.to(device)

/Users/shelger/miniconda3/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [3]:
import pandas as pd
train_data = pd.read_csv('TrainingData/jigsaw-toxic-comment-train-processed-seqlen128.csv.zip')
print(train_data.head())

/Users/shelger/miniconda3/envs/langchain/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  \
0             0        0       0       0              0   
1             0        0       0       0              0   
2             0        0       0       0              0   
3             0        0       0       0              0   
4             0        0       0       0              0   

                                      input_word_ids  \
0  (101, 27746, 31609, 11809, 24781, 10105, 70971...   
1  (101, 141, 112, 56237, 10874, 106, 10357, 1

In [4]:
train_data_unintended = pd.read_csv('TrainingData/jigsaw-unintended-bias-train-processed-seqlen128.csv.zip')
print(train_data_unintended.head())

      id                                       comment_text     toxic  \
0  59848  This is so cool. It's like, 'would you want yo...  0.000000   
1  59849  Thank you!! This would make my life a lot less...  0.000000   
2  59852  This is such an urgent design problem; kudos t...  0.000000   
3  59855  Is this something I'll be able to install on m...  0.000000   
4  59856               haha you guys are a bunch of losers.  0.893617   

   severe_toxicity  obscene  identity_attack   insult  threat  asian  atheist  \
0         0.000000      0.0         0.000000  0.00000     0.0    NaN      NaN   
1         0.000000      0.0         0.000000  0.00000     0.0    NaN      NaN   
2         0.000000      0.0         0.000000  0.00000     0.0    NaN      NaN   
3         0.000000      0.0         0.000000  0.00000     0.0    NaN      NaN   
4         0.021277      0.0         0.021277  0.87234     0.0    0.0      0.0   

   ...  wow  sad  likes  disagree  sexual_explicit  identity_annotator_cou

In [5]:
print(train_data['all_segment_id'][12])

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


In [6]:
from torch.utils.data import Dataset

class ToxicCommentsDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor([int(x) for x in self.df.iloc[idx]['input_word_ids'].strip('()').split(', ')]),
            'attention_mask': torch.tensor([int(x) for x in self.df.iloc[idx]['input_mask'].strip('()').split(', ')]),
            'token_type_ids': torch.tensor([int(x) for x in self.df.iloc[idx]['all_segment_id'].strip('()').split(', ')])
        }
        label_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
        labels = [self.df.iloc[idx][col] for col in label_columns]
        item['labels'] = torch.tensor(labels, dtype=torch.float)
        return item

    def __len__(self):
        return len(self.df)

In [7]:
from torch.utils.data import DataLoader
dataset = ToxicCommentsDataset(train_data)
dataloader = DataLoader(dataset, batch_size=512, shuffle=True)

In [8]:
for name, param in model.named_parameters():
    print(name)
    if 'classifier' not in name:
        param.requires_grad = False   

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

In [9]:
from transformers import AdamW
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)

/Users/shelger/miniconda3/envs/langchain/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
validation_data = pd.read_csv('TrainingData/validation-processed-seqlen128.csv.zip')
print(len(validation_data))

In [11]:
from torch.nn import BCEWithLogitsLoss
loss_fn = BCEWithLogitsLoss()
model.train()
for batch in dataloader:
    inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
    labels = batch['labels'].to(device)
    outputs = model(**inputs)
    # print("Logits shape:", outputs.logits.shape)
    # print("Labels shape:", labels.shape)

    loss = loss_fn(outputs.logits, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(f"Loss: {loss.item()}")

Loss: 0.7131073474884033
Loss: 0.703738808631897
Loss: 0.7008733749389648
Loss: 0.6925168037414551
Loss: 0.6849724650382996
Loss: 0.6840934157371521
Loss: 0.6771616339683533
Loss: 0.6697148680686951
Loss: 0.6653908491134644
Loss: 0.6618555784225464
Loss: 0.6523333787918091
Loss: 0.6482284069061279
Loss: 0.6431896090507507
Loss: 0.6364555358886719
Loss: 0.6320183277130127
Loss: 0.6240700483322144
Loss: 0.62134850025177
Loss: 0.6148040294647217
Loss: 0.6131453514099121
Loss: 0.6045658588409424
Loss: 0.6000387072563171
Loss: 0.5931602716445923
Loss: 0.5878549814224243
Loss: 0.5890147686004639
Loss: 0.5776566863059998
Loss: 0.5751050710678101
Loss: 0.5696629881858826
Loss: 0.5635476112365723
Loss: 0.5607060194015503
Loss: 0.5568093061447144
Loss: 0.5506417155265808
Loss: 0.5474492311477661
Loss: 0.5433719158172607
Loss: 0.5375375151634216
Loss: 0.5296816229820251
Loss: 0.5274196267127991
Loss: 0.5269730091094971
Loss: 0.5220991373062134
Loss: 0.5094466209411621
Loss: 0.5126747488975525
Los

In [37]:
# test
test_data = pd.read_csv('TrainingData/validation-processed-seqlen128.csv.zip')
print(test_data)

        id                                       comment_text lang  toxic  \
0        0  Este usuario ni siquiera llega al rango de    ...   es      0   
1        1  Il testo di questa voce pare esser scopiazzato...   it      0   
2        2  Vale. Sólo expongo mi pasado. Todo tiempo pasa...   es      1   
3        3  Bu maddenin alt başlığı olarak  uluslararası i...   tr      0   
4        4  Belçika nın şehirlerinin yanında ilçe ve belde...   tr      0   
...    ...                                                ...  ...    ...   
7995  7995   Il fatto è che la pagina dei personaggi minor...   it      0   
7996  7996  El imbesil ete dela luna no se entera ni ostia...   es      1   
7997  7997  olum sız manyakmısınz siz adam sıze sanal yıld...   tr      1   
7998  7998  El mapa del reinado de Alhaken esta ligerament...   es      0   
7999  7999  lasciami la tua email per favore. ad ogni modo...   it      0   

                                         input_word_ids  \
0     (101, 1251

In [12]:
torch.save(model.state_dict(), 'finetuned_bert.pth')

In [21]:
# model.load_state_dict(torch.load('model_state_dict.pth'))
# here i am going to use validation model to check the accuracy of my model:
print(validation_data.columns)
print(train_data.columns)

Index(['id', 'comment_text', 'lang', 'toxic', 'input_word_ids', 'input_mask',
       'all_segment_id'],
      dtype='object')
Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate', 'input_word_ids', 'input_mask',
       'all_segment_id'],
      dtype='object')


In [34]:
class TestToxicCommentsDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor([int(x) for x in self.df.iloc[idx]['input_word_ids'].strip('()').split(', ')]),
            'attention_mask': torch.tensor([int(x) for x in self.df.iloc[idx]['input_mask'].strip('()').split(', ')]),
            'token_type_ids': torch.tensor([int(x) for x in self.df.iloc[idx]['all_segment_id'].strip('()').split(', ')])
        }
        return item

    def __len__(self):
        return len(self.df)

In [39]:
test_dataset = TestToxicCommentsDataset(validation_data)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)

In [46]:
import numpy as np
def evaluate_model(threshold=0.5):
    model.eval()
    all_predictions = []
    all_labels = validation_data['toxic'].values

    with torch.no_grad():
        for batch in test_dataloader:
            print("batch working")
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            
            outputs = model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.sigmoid(logits).cpu().numpy()[:, 0]
            all_predictions.extend(predictions)
    
    predicted_labels = (np.array(all_predictions) > threshold).astype(int)
    
    correct_predictions = (predicted_labels == all_labels).sum()
    accuracy = correct_predictions / len(all_labels)
    return accuracy

In [47]:
accuracy = evaluate_model()
print(f"Model accuracy on test data: {accuracy:.4f}")

batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
batch working
Model accuracy on test data: 0.8462
